# WEB: Quantum Teleportation Protocol

Learn how to teleport quantum states using entanglement and classical communication!

Quantum teleportation is one of the most famous quantum protocols. It allows you to transfer an unknown quantum state from one location to another using:
1. SHINE: **Entanglement** (shared between sender and receiver)
2. SIGNAL: **Classical communication** (measurement results)
3. TARGET: **Local operations** (corrections based on measurements)

The amazing part: the original state is destroyed in the process - true teleportation!

In [ ]:
from quantumsim import Circuit, Executor
from quantumsim.viz.ascii import ASCIIDrawer
import numpy as np

print("WEB: Quantum Teleportation Protocol")
print("=" * 35)

executor = Executor()

## Step 1: Prepare the Unknown State

Alice has a qubit in an unknown state |ψ that she wants to teleport to Bob.
Let's create a specific state to teleport (but pretend Alice doesn't know what it is!):

In [ ]:
# Create an unknown state |ψ = α|0 + β|1
# Let's use |+ = (|0 + |1)/√2 as our secret state

unknown_state = Circuit(1)
unknown_state.h(0) # Creates |+ state

print("DEMO: Unknown state to teleport:")
print(ASCIIDrawer(unknown_state).draw())

# Verify what state we're teleporting
state_to_teleport = executor.run(unknown_state)
print(f"State amplitudes: {state_to_teleport.amplitudes}")
print("→ This is the |+ state: equal superposition!")

## Step 2: Create Entangled Pair

Alice and Bob share an entangled pair (Bell state).
Alice keeps qubit 1, Bob gets qubit 2:

In [ ]:
# Create shared entanglement between Alice and Bob
entanglement = Circuit(2)
entanglement.h(0) # Alice's half
entanglement.cx(0, 1) # Entangle with Bob's half

print("LINK: Shared entanglement:")
print(ASCIIDrawer(entanglement).draw())

entangled_state = executor.run(entanglement)
measurements = entangled_state.measure_all(shots=100)
print(f"Entangled measurements: {measurements}")
print("→ Perfect correlation: only |00 and |11!")

## Step 3: Full Teleportation Circuit

Now let's build the complete teleportation protocol:
- Qubit 0: Alice's unknown state |ψ
- Qubit 1: Alice's half of entangled pair 
- Qubit 2: Bob's half of entangled pair

In [ ]:
# Complete teleportation circuit
teleport = Circuit(3)

# Step 1: Prepare unknown state on qubit 0
teleport.h(0) # |ψ = |+

# Step 2: Create entanglement between qubits 1 and 2
teleport.h(1)
teleport.cx(1, 2)

# Step 3: Alice performs Bell measurement on qubits 0 and 1
teleport.cx(0, 1) # Entangle unknown state with Alice's half
teleport.h(0) # Complete Bell measurement

print("SIGNAL: Full Teleportation Circuit:")
print(ASCIIDrawer(teleport).draw())

## Step 4: Simulate the Protocol

Let's run the teleportation and see what happens!

In [ ]:
# Run teleportation protocol
final_state = executor.run(teleport)

print("TARGET: Teleportation Results:")
print("=" * 25)

# Measure Alice's qubits (0 and 1) - these are the "classical bits"
measurements = final_state.measure_all(shots=1000)

# Count different measurement outcomes
outcomes = {}
for result, count in measurements.items():
 alice_bits = result[:2] # First two bits (Alice's measurement)
 bob_bit = result[2] # Last bit (Bob's qubit)
 
 if alice_bits not in outcomes:
 outcomes[alice_bits] = []
 outcomes[alice_bits].append((bob_bit, count))

print("Alice's measurement → Bob's qubit state:")
for alice_result, bob_outcomes in outcomes.items():
 total_count = sum(count for _, count in bob_outcomes)
 print(f"\nAlice measures {alice_result}:")
 for bob_bit, count in bob_outcomes:
 probability = count / total_count
 print(f" Bob's qubit in |{bob_bit}: {probability:.3f} ({count}/{total_count})")

## Step 5: Understanding the Results

Based on Alice's measurement results, Bob needs to apply corrections:

| Alice's Result | Bob's Correction | Final State |
|---------------|------------------|-------------|
| 00 | Do nothing | |ψ |
| 01 | Apply X gate | |ψ |
| 10 | Apply Z gate | |ψ |
| 11 | Apply ZX gates | |ψ |

Let's implement the corrections:

In [ ]:
def teleport_with_correction(unknown_state_prep):
 """
 Complete teleportation with Bob's corrections
 """
 print(f"DEMO: Teleporting: {unknown_state_prep.__name__}")
 
 # Build circuit
 teleport = Circuit(3)
 
 # Prepare unknown state
 unknown_state_prep(teleport, 0)
 
 # Create entanglement
 teleport.h(1)
 teleport.cx(1, 2)
 
 # Alice's Bell measurement
 teleport.cx(0, 1)
 teleport.h(0)
 
 # Simulate and get one outcome
 state = executor.run(teleport)
 result = list(state.measure_all(shots=1).keys())[0]
 
 alice_result = result[:2]
 print(f"SIGNAL: Alice measures: {alice_result}")
 
 # Bob applies corrections based on Alice's result
 correction_circuit = Circuit(1)
 if alice_result[1] == '1': # Apply X if second bit is 1
 correction_circuit.x(0)
 print("CONFIG: Bob applies X correction")
 if alice_result[0] == '1': # Apply Z if first bit is 1
 correction_circuit.z(0)
 print("CONFIG: Bob applies Z correction")
 
 # Show final state (Bob's qubit after correction)
 bob_final = int(result[2])
 print(f"TARGET: Bob's final state: |{bob_final}")
 
 return alice_result, bob_final

# Test with different states
def prepare_plus_state(circuit, qubit):
 circuit.h(qubit)

def prepare_minus_state(circuit, qubit):
 circuit.x(qubit)
 circuit.h(qubit)

def prepare_zero_state(circuit, qubit):
 pass # Already |0

print("Testing teleportation with different states:")
print("=" * 45)

teleport_with_correction(prepare_plus_state)
print()
teleport_with_correction(prepare_minus_state)
print()
teleport_with_correction(prepare_zero_state)

## EXPERIMENT: Interactive Exercise

Try teleporting your own quantum states! 

Modify the function below to prepare any state you want, then see how teleportation preserves it:

In [ ]:
def prepare_my_state(circuit, qubit):
 """
 Prepare your custom quantum state here!
 Examples:
 - circuit.h(qubit) # |+ state
 - circuit.x(qubit) # |1 state 
 - circuit.ry(np.pi/3, qubit) # Custom superposition
 """
 # Your state preparation here:
 circuit.h(qubit) # Try changing this!
 
# Test your state
print("EXPERIMENT: Testing your custom state:")
teleport_with_correction(prepare_my_state)

## SUCCESS: Key Takeaways

1. **No Cloning**: The original state is destroyed during measurement
2. **Classical Communication Required**: Alice must tell Bob her measurement results
3. **Perfect Fidelity**: When done correctly, the state is perfectly transferred
4. **Entanglement is Key**: The shared entangled pair makes it all possible
5. **Corrections Matter**: Bob must apply the right gates based on Alice's measurements

SPARKLE: **Amazing fact**: This protocol works for ANY unknown quantum state, not just the examples we tried!

You can experiment with different initial states in the exercise above!